In [1]:
import serial
import time
import re

PORTA = 'COM8'
BAUDRATE = 1200
BYTESIZE = serial.SEVENBITS
PARITY = serial.PARITY_ODD
STOPBITS = serial.STOPBITS_ONE
TIMEOUT = 0.5 # Timeout menor para o loop girar mais rapido

def conectar_sartorius():
    try:
        ser = serial.Serial(
            port=PORTA,
            baudrate=BAUDRATE,
            bytesize=BYTESIZE,
            parity=PARITY,
            stopbits=STOPBITS,
            timeout=TIMEOUT,
            rtscts=False  # Deixe False para gerenciarmos manualmente
        )
        
        # --- ADICIONE ISTO AQUI ---
        ser.rts = True  # Liga o pino 7 (Diz para a balança: "Pode mandar!")
        ser.dtr = True  # Liga o pino 4 (Diz para a balança: "Estou ligado!")
        # --------------------------

        print(f"Conectado à Sartorius CCE na {PORTA}")
        return ser
    except serial.SerialException as e:
        print(f"Erro de conexão: {e}")
        return None

def ler_peso(ser):
    if ser.in_waiting > 0:
        # Lê tudo que tem lá
        raw_data = ser.readline()
        try:
            # Tenta decodificar para ver o que está chegando
            texto = raw_data.decode('ascii', errors='ignore').strip()
            
            # Se vier vazio, ignora
            if not texto: return None

            print(f"Bruto recebido: '{texto}'") # Debug para ver o formato real

            match = re.search(r"[-+]?\s*\d+\.\d+", texto)
            if match:
                peso_str = match.group().replace(" ", "")
                return float(peso_str)
        except Exception as e:
            print(f"Erro no parse: {e}")
    return None

# --- LOOP PRINCIPAL ---
if __name__ == "__main__":
    balanca = conectar_sartorius()
    
    if balanca:
        try:
            print("Iniciando leitura. Pressione Ctrl+C para parar.")
            
            while True:
                balanca.write(b'\x1bP\r\n') 
                
                peso = ler_peso(balanca)
                
                if peso is not None:
                    print(f"--- PESO DETECTADO: {peso} g ---")
                
                time.sleep(0.5) # Espera um pouco

        except KeyboardInterrupt:
            print("Parando...")
            balanca.close()

Conectado à Sartorius CCE na COM8
Iniciando leitura. Pressione Ctrl+C para parar.
Bruto recebido: '+   0.0016 g'
--- PESO DETECTADO: 0.0016 g ---
Bruto recebido: '+   0.0016 g'
--- PESO DETECTADO: 0.0016 g ---
Bruto recebido: '+   0.0016 g'
--- PESO DETECTADO: 0.0016 g ---
Bruto recebido: '+   0.0016 g'
--- PESO DETECTADO: 0.0016 g ---
Bruto recebido: '+   0.0016 g'
--- PESO DETECTADO: 0.0016 g ---
Bruto recebido: '+   0.0016 g'
--- PESO DETECTADO: 0.0016 g ---
Bruto recebido: '+   0.0014 g'
--- PESO DETECTADO: 0.0014 g ---
Bruto recebido: '+   0.0014 g'
--- PESO DETECTADO: 0.0014 g ---
Bruto recebido: '+   0.0014 g'
--- PESO DETECTADO: 0.0014 g ---
Bruto recebido: '+   0.0014 g'
--- PESO DETECTADO: 0.0014 g ---
Bruto recebido: '+   0.0014 g'
--- PESO DETECTADO: 0.0014 g ---
Bruto recebido: '+   0.0014 g'
--- PESO DETECTADO: 0.0014 g ---
Bruto recebido: '+   0.0014 g'
--- PESO DETECTADO: 0.0014 g ---
Parando...


In [2]:
import serial
import time

# Ajuste para sua porta
PORTA = 'COM8' 

try:
    ser = serial.Serial(PORTA, 9600, timeout=1)
    print(f"--- TESTE DE LOOPBACK NA {PORTA} ---")
    print("Enviando: 'TESTE'")
    
    ser.write(b'TESTE')
    time.sleep(0.5)
    
    recebido = ser.read(ser.in_waiting)
    print(f"Recebido: {recebido}")
    
    if b'TESTE' in recebido:
        print("\nSUCESSO: O cabo e o adaptador USB estão funcionando!")
    else:
        print("\nFALHA: O sinal não voltou. Problema no cabo ou no adaptador USB.")

    ser.close()
except Exception as e:
    print(f"Erro: {e}")

--- TESTE DE LOOPBACK NA COM8 ---
Enviando: 'TESTE'
Recebido: b''

FALHA: O sinal não voltou. Problema no cabo ou no adaptador USB.
